## Importando bibliotecas

In [2]:
import pyodbc
import pandasql as ps
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats

## Conectando ao database

Vamos primeiro conectar ao database que vamos utilizar neste exercicio. Note que, se você utilizar os mesmos nomes que estão em "Server" e "Database" em seu computador, não vai funcionar. Você deve altera-los para seu ambiente especifico.

In [3]:
dados_conexao = (r"Driver={SQL Server};"
                 r"Server=DESKTOP-2TIOC7M\SQLEXPRESS;" 
                 r"Database=Project4Database;"
                 r"Trusted_Connection=yes;")

In [4]:
conexao = pyodbc.connect(dados_conexao)

In [5]:
cursor = conexao.cursor()

## Queries e testes de hipóteses

Vamos agora, com a conexão feita, utilizar  queries semelhantes ao  do exercicio anterior para ver os dados visualmente e testar hipóteses. <br> Antes, vimos o status como "Current" para quem tem asma, "Former" para quem já teve asma  e se "curou", e "never" para que nunca teve asma. Vamos simplificar um pouquinho: vamos só considerar que tem asma atualmente ou não, através da variavel "CASTHM1"( "Adults who have been told they currently have asthma").

### Ter asma/Não ter asma VS Morar em zonas rurais/urbanas

In [36]:
df_Urb_Asth = pd.read_sql("""-- Pergunta inicial  1-	Uma maior proporção de pessoas que moram em cidade é asmáticas quando comparadas aquelas que moram em zonas rurais?

-- Não , proporções são muito parecidas.
SELECT 
URBSTAT,
CASE 
	WHEN URBSTAT = 1  THEN 'Urban'
	WHEN URBSTAT = 2  THEN 'Rural'
	ELSE NULL END AS  URBSTAT_Categories,
CASTHM1,
CASE 
	WHEN CASTHM1  =  1 THEN  'No'
	WHEN CASTHM1  =  2 THEN  'Yes'
	ELSE NULL END AS CASTHM1_Categories,
COUNT(*) AS Count,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY URBSTAT),2) AS Percentage
FROM AsthmaStatus 
INNER JOIN UrbanStatus ON AsthmaStatus.ID = UrbanStatus.ID
WHERE CASTHM1 IS NOT NULL
AND URBSTAT IS NOT NULL
GROUP BY URBSTAT,CASTHM1
ORDER BY CASTHM1, URBSTAT;""", conexao)
df_Urb_Asth

,URBSTAT,URBSTAT_Categories,CASTHM1,CASTHM1_Categories,Count,Percentage
0,1,Urban,1,No,330273,90.13
1,2,Rural,1,No,55832,90.51
2,1,Urban,2,Yes,36154,9.87
3,2,Rural,2,Yes,5853,9.49


Esta funcionando, agora vamos para a visulização de dados e testes estatísticos para cada hipótese

In [37]:
px.bar(df_Urb_Asth, x="CASTHM1_Categories", y = "Percentage", facet_col= "URBSTAT_Categories" )


Aparentemente, não parecem ser dependentes, porém vamos checar isso com um teste de hipótese.
Vamos simplificar um pouco, vamos tentar ver se tem alguma dependência entre apenas ter asma e morar em um local urbano/rural.

In [18]:
Urb_Asth_contigency = df_Urb_Asth["Count"]
Urb_Asth_contigency = [[Urb_Asth_contigency[0], Urb_Asth_contigency[2] + Urb_Asth_contigency[4]],
                       [Urb_Asth_contigency[1], Urb_Asth_contigency[3] + Urb_Asth_contigency[5]]]
Urb_Asth_contigency = pd.DataFrame(Urb_Asth_contigency, columns=["Com asma", "Sem Asma"], index = ["Vivem em ambientes urbanos", "Vivem em ambientes rurais"])
Urb_Asth_contigency

,Com asma,Sem Asma
Vivem em ambientes urbanos,36154,330273
Vivem em ambientes rurais,5853,55832


In [19]:
stats.chi2_contingency(pd.DataFrame(Urb_Asth_contigency)
)

Chi2ContingencyResult(statistic=8.486600112886542, pvalue=0.003577717710792795, dof=1, expected_freq=array([[ 35954.37406333, 330472.62593667],
       [  6052.62593667,  55632.37406333]]))

Diferente do que haviamos proposto aparentemente, o teste de hipótese aponta que são sim dependentes, porém deve-se olhar para os resultados com devida cautela, dado que normalmente em amostras muito grandes, é comum os valores de p serem menores. Portanto, vamos ver a magnitude dessa dependência através de um Odds ratio.

In [20]:
stats.contingency.odds_ratio(pd.DataFrame(Urb_Asth_contigency)


OddsRatioResult(statistic=1.0442101256289011)

Como pode-se observar a existe 1.04 mais chances de você ter asma se você vive em um ambiente urbano do que em um ambiente rural, o que, na prática, é praticamente insignificante.

### Ter asma/Não ter asma VS Morar em zonas Metropolitanas/Não-metropolitanas

In [26]:
df_Met_Asth = pd.read_sql("""
SELECT 
METSTAT,
CASE 
	WHEN METSTAT = 1  THEN 'Metropolitan'
	WHEN METSTAT = 2  THEN 'Non-metropolitan'
	ELSE NULL END AS  METSTAT_Categories,
ASTHMS1,
CASE 
	WHEN ASTHMS1  =  1 THEN  'Current'
	WHEN ASTHMS1  =  2 THEN  'Former'
	WHEN ASTHMS1 = 3  THEN 'Never'
	ELSE NULL END AS ASTHMS1_Categories,
COUNT(*) AS Count,
ROUND(COUNT (*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY METSTAT),2) AS Percentage
FROM AsthmaStatus 
INNER JOIN UrbanStatus
ON AsthmaStatus.ID = UrbanStatus.ID
WHERE ASTHMS1 IS NOT NULL
AND METSTAT IS NOT NULL
GROUP BY METSTAT,ASTHMS1
ORDER BY ASTHMS1,METSTAT;""", conexao)
df_Met_Asth

,METSTAT,METSTAT_Categories,ASTHMS1,ASTHMS1_Categories,Count,Percentage
0,1,Metropolitan,1,Current,29363,9.86
1,2,Non-metropolitan,1,Current,12644,9.71
2,1,Metropolitan,2,Former,12085,4.06
3,2,Non-metropolitan,2,Former,4569,3.51
4,1,Metropolitan,3,Never,256439,86.09
5,2,Non-metropolitan,3,Never,113012,86.78


In [27]:
px.bar(df_Met_Asth, x="ASTHMS1_Categories", y = "Percentage", facet_col= "METSTAT_Categories" )

###

In [29]:
Met_Asth_contigency = df_Met_Asth["Count"]
Met_Asth_contigency = [[Met_Asth_contigency[0], Met_Asth_contigency[2] + Met_Asth_contigency[4]],
                       [Met_Asth_contigency[1], Met_Asth_contigency[3] + Met_Asth_contigency[5]]]
Met_Asth_contigency = pd.DataFrame(Met_Asth_contigency, columns=["Com asma", "Sem Asma"], index = ["Vivem em ambientes metropolitanos", "Vivem em ambientes não-metropolitanos"])
Met_Asth_contigency

,Com asma,Sem Asma
Vivem em ambientes metropolitanos,29363,268524
Vivem em ambientes não-metropolitanos,12644,117581


In [31]:
stats.chi2_contingency(pd.DataFrame(Met_Asth_contigency))

Chi2ContingencyResult(statistic=2.2184382491202173, pvalue=0.13637094699573296, dof=1, expected_freq=array([[ 29229.12510978, 268657.87489022],
       [ 12777.87489022, 117447.12510978]]))

Wow, dessa vez nossa hipótese foi confirmada, não parece que há uma dependência entre morar em um ambiente metropolitano/não-metropolitano e ter asma ou não. Logo, não precisamos calcular o odds ratio, pois provavelmente, o intervalo de confiança em que o mesmo vai pertencer vai incluir o 1, o que indica que não há dependência.